In [29]:
from together import Together
import os
from dotenv import load_dotenv
import pandas as pd
import re
import numpy as np
import json
from tqdm import tqdm
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score
load_dotenv()
client = Together(api_key=os.environ.get("TOGETHER_API_KEY"))


In [12]:
df_detection = pd.read_csv('silent_signals_detection.csv')
df_disambiguation = pd.read_csv('silent_signals_disambiguation.csv')

### Detection

In [ ]:
def llm_detection(sentence):
    input_definition = f"""You are a helpful assistant participating in a content moderation task of dog whistle detection.

    A dog whistle is the use of coded or suggestive language in political messaging to garner support from a particular group without provoking opposition. 
    Does the following sentence contain a dog whistle?
    If it does, please identify the dog whistle.

    {sentence}
    
    Please structure your response as a JSON object,
    where {{
      "sentence": "<original sentence here>",
      "dog_whistle_present": <true or false>,
      "dog_whistle_term": "<term if present, else null>",
      "coded_meaning": "<coded meaning if present, else null>"
    }}.
    """
    response = client.chat.completions.create(
        model="deepseek-ai/DeepSeek-R1-Distill-Llama-70B-free",
        messages=[{"role": "user", "content": input_definition}]
    )
    raw_output = response.choices[0].message.content
    print('done')
    match = re.search(r"```json\s*(.*?)```", raw_output, flags=re.DOTALL)
    cleaned = match.group(1) if match else ""
    return cleaned 

In [19]:
results = []
for index, row in df_detection[:30].iterrows():
    json_result = llm_detection(row['example'])
    try:
      response_dict = json.loads(json_result)
      results.append(response_dict)
    except json.JSONDecodeError as e:
      print(f"Error decoding JSON at row {index}: {e}") 
with open('deepseekr1.json', 'w') as outfile:
  json.dump(results, outfile, indent=4)

done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done


In [33]:
with open('deepseekr1.json', 'r') as f:
  data = json.load(f)
with open('deepseekr1_collab.json', 'r') as f:
  data1 = json.load(f)
  
detection_outputs = pd.concat([pd.DataFrame(data), pd.DataFrame(data1)], ignore_index=True)
detection_outputs = pd.concat([detection_outputs, df_detection], axis=1)
detection_outputs['dog_whistle_present'] = detection_outputs['dog_whistle_present'].map({True:'coded', False:'noncoded'})
detection_outputs['label'] = detection_outputs['label'].map({'noncoded':'noncoded', 'coded':'coded', 'non-coded':'noncoded'})
detection_outputs

,sentence,dog_whistle_present,dog_whistle_term,coded_meaning,dog_whistle,dog_whistle_root,ingroup,definition,example,label
0,Might I suggest you reach out to YWNBAW? Very ...,coded,YWNBAW,You Will Not Be A Winner,YWNBAW,YWNBAW,transphobic,You Will Never Be A Woman (YWNBAW) is an alt-r...,"""Might I suggest you reach out to YWNBAW? Very...",coded
1,"Wow 109 countries, and they consider us as cat...",coded,blood libel and black cube of saturn,"References to antisemitic conspiracy theories,...",109,109,antisemitic,109 stands for the 109 locations that Jews wer...,"""""Wow 109 countries, and they consider us as ...",coded
2,A mountain of evidence shows climate alarmists...,coded,climate alarmists,The term 'climate alarmists' is a coded way to...,alarmist,alarmist,climate change denier,"""Alarmist"" is a common global warming denialis...","""A mountain of evidence shows climate alarmist...",coded
3,TRAs always need to get the last word in and n...,coded,TRAs,"Refers to Trans Rights Activists or Advocates,...",Aiden,Aiden,transphobic,"Derogatory term for a transgender man, based o...","""TRAs always need to get the last word in and ...",coded
4,We are stopping cold the attacks on Judeo-Chri...,coded,"Judeo-Christian values, Merry Christmas",Appealing to conservative or religious groups ...,Judeo-Christian,Judeo-Christian,Islamophobic,"It might seem neighbourly, even pluralistic, t...","""We are stopping cold the attacks on Judeo-Chr...",coded
...,...,...,...,...,...,...,...,...,...,...
96,"As the plane accelerates, we feel like we’re b...",noncoded,None,None,norm20,NaN,NaN,NaN,"As the plane accelerates, we feel like we’re b...",noncoded
97,"Depending on the type of bomb, weapon safety r...",noncoded,None,None,norm21,NaN,NaN,NaN,"Depending on the type of bomb, weapon safety r...",noncoded
98,"and to ad on, even humans don't die of 'old ag...",noncoded,None,None,norm22,NaN,NaN,NaN,"and to ad on, even humans don't die of ""old ag...",noncoded
99,I just don't like it anymore. I lived in Wisco...,noncoded,None,None,norm23,NaN,NaN,NaN,I just don't like it anymore. I lived in Wisco...,noncoded


In [34]:
print(accuracy_score(detection_outputs['label'], detection_outputs['dog_whistle_present']))
print(f1_score(detection_outputs['label'], detection_outputs['dog_whistle_present'], pos_label='coded'))
print(recall_score(detection_outputs['label'], detection_outputs['dog_whistle_present'], pos_label='coded'))
print(precision_score(detection_outputs['label'], detection_outputs['dog_whistle_present'], pos_label='coded'))

0.7920792079207921
0.8292682926829268
1.0
0.7083333333333334


In [44]:
from sentence_transformers import SentenceTransformer
from scipy.spatial.distance import cosine


# Load model
model = SentenceTransformer('all-MiniLM-L6-v2')
detection_outputs_coded = detection_outputs[detection_outputs['label'] == 'coded']

# Get embeddings
coded_embeddings = model.encode(detection_outputs_coded['coded_meaning'].tolist(), convert_to_numpy=True)
definition_embeddings = model.encode(detection_outputs_coded['definition'].tolist(), convert_to_numpy=True)

# Initialize similarity/distance lists
cosine_sims = []
l2_dists = []
l1_dists = []

# Loop over pairs
for c, d in zip(coded_embeddings, definition_embeddings):
    cosine_sims.append(1 - cosine(c, d))
    l2_dists.append(np.linalg.norm(c - d))          # Euclidean
    l1_dists.append(np.sum(np.abs(c - d)))          # Manhattan

# Compute averages
average_cosine_sim = np.mean(cosine_sims)
average_l2_dist = np.mean(l2_dists)
average_l1_dist = np.mean(l1_dists)

# Output
print("Average cosine similarity:", average_cosine_sim)
print("Average L2 distance (Euclidean):", average_l2_dist)
print("Average L1 distance (Manhattan):", average_l1_dist)


Average cosine similarity: 0.4408473
Average L2 distance (Euclidean): 1.0461692
Average L1 distance (Manhattan): 16.31832


### Disambiguation


In [ ]:
def llm_disambiguation(lst_sentences, dog_whistle, definition_of_dog_whistle):
    input_disambiguation =  f"""You are a helpful assistant participating in a content moderation task of dog whistle detection.
                            A dogwhistle is the use of coded or suggestive language in political messaging to garner support 
                            from a particular group without provoking opposition. 
                            The coded meaning of {dog_whistle} is: {definition_of_dog_whistle}.  Can you identify which sentences in the set below are using {dog_whistle} as a dog whistle? {lst_sentences}"""
    response = client.chat.completions.create(
        model="deepseek-ai/DeepSeek-R1-Distill-Llama-70B-free",
        messages=[{"role": "user", "content": input_disambiguation}]
    )
    raw_output = response.choices[0].message.content
    # Remove <think>...</think> and strip whitespace
    clean_output = re.sub(r"<think>.*?</think>", "", raw_output, flags=re.DOTALL).strip()
    return clean_output 


In [64]:
dict_dog_whistle = {}
for dog_whistle in np.unique(df_disambiguation['dog_whistle']):
    print(dog_whistle)
    definition_of_dog_whistle = list(df_disambiguation.loc[df_disambiguation['dog_whistle']==dog_whistle, 'definition'])[0]
    lst_sentences = list(df_disambiguation.loc[df_disambiguation['dog_whistle']==dog_whistle, 'content'])
    dict_dog_whistle[dog_whistle] = llm_disambiguation(lst_sentences, dog_whistle, definition_of_dog_whistle)

Baltimore
Islamic extremism
Islamic terrorism
based
echo
election integrity
extremism
extremist
extremists
fatherless
fatherless home
globalist
globalists
grooming
hygienic
jogger
law and order
soy
steroids
terrorist
welfare


In [65]:
import pickle
with open('deepseek_r1_distill_llama_disambiguation.pkl', 'wb') as f:
    pickle.dump(dict_dog_whistle, f)

In [66]:
dict_dog_whistle

{'Baltimore': 'The sentences that use "Baltimore" as a dog whistle are:\n\n1. "This is George Washington. Another Washington statue was defaced in Baltimore. A statue of Thomas Jefferson was ripped down in Portland also."\n\n3. "This is happening throughout our country. We have had gun violence in Baltimore. A few years ago, we had gun violence at Great Mills High School in Maryland."\n\nThese sentences subtly evoke stereotypes about crime and minorities in Baltimore through references to statue defacement and gun violence, respectively.',
 'Islamic extremism': 'The sentence in question does not use Islamic extremism as a dog whistle. It specifically refers to involvement in organizations promoting extremism, avoiding any generalization about all Muslims. Therefore, it does not imply that Muslim people as a whole are a threat.\n\n**Answer:** The sentence does not use Islamic extremism as a dog whistle. It targets extremist organizations without generalizing about all Muslims.',
 'Islam